## load libraries

In [12]:
import torch 
import torch.nn as nn 
from torch.optim import optimizer
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Preprocessing

In [30]:
transform =transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

## load dataset

In [31]:
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


## build architecture

In [35]:
class cnn_Cifar10(nn.Module):
    def __init__(self):
        super(cnn_Cifar10, self).__init__()

        #feature extractor
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.cnn2 = nn.Conv2d(6, 16, 3)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        #classifier
        self.fc1 = nn.Linear(6 * 6 * 16, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 10)

    def forward(self, x):
        #feature extractor
        x = self.cnn1(x)
        x = nn.ReLU()(x)
        x = self.pool1(x)

        x=self.pool2(nn.ReLU()(self.cnn2(x)))
        
        #flatten
        x=self.fc1(x)
        x = nn.ReLU()(x)
        x=self.fc2(x)
        x = nn.ReLU()(x)
        x=self.fc3(x)
        x = nn.ReLU()(x)
        x=self.fc4(x)
        return x
    
        

## create object

In [36]:
model = cnn_Cifar10()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## train

In [34]:
for epoch in range(10):
    for images, labels in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Epoch [{epoch+1}/10], Step [{images+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
for images, labels in enumerate(test_loader):
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')
    
        

TypeError: conv2d() received an invalid combination of arguments - got (int, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!int!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!int!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
